In [2]:
!conda info



     active environment : base
    active env location : C:\Users\tj_da\anaconda3
            shell level : 1
       user config file : C:\Users\tj_da\.condarc
 populated config files : C:\Users\tj_da\.condarc
          conda version : 23.1.0
    conda-build version : 3.23.3
         python version : 3.10.9.final.0
       virtual packages : __archspec=1=x86_64
                          __win=0=0
       base environment : C:\Users\tj_da\anaconda3  (writable)
      conda av data dir : C:\Users\tj_da\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/noarch
          package cache : C:\Users\

In [1]:
# Import libraries needed
import csv
import sqlite3
import json
import pandas as pd
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, Column, Integer, String, Date, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os



ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# def create_database_and_import_csv(database_name, table_name, csv_file):
#     # Read the CSV file to infer column types and names
#     df = pd.read_csv(csv_file, nrows=1)

#     # Connect to SQLite database
#     conn = sqlite3.connect(database_name)
#     cursor = conn.cursor()

#     # Create a table in the database based on CSV data types
#     create_table_query = f'''
#         CREATE TABLE {table_name} (
#             {', '.join(['{} {}'.format(column, df[column].dtype) for column in df.columns])}
#         )
#     '''
#     cursor.execute(create_table_query)

#     # Import CSV data into the table
#     df.to_sql(table_name, conn, if_exists='replace', index=False)

#     # Commit the changes and close the connection
#     conn.commit()
#     conn.close()



In [ ]:
# # Create SQLite Database, Table, and Import CSV Data

# database_name = 'drug_side_effects.db'
# table_name = 'drug_table'
# csv_file = 'Data/drugs_side_effects_drugs_com.csv'


# create_database_and_import_csv(database_name, table_name, csv_file)


In [ ]:
# def plot_bar_graph(database_name, table_name, column_to_plot):
#     # Connect to SQLite database
#     conn = sqlite3.connect(database_name)

#     # Query data from the database into a DataFrame
#     query = f'SELECT * FROM {table_name}'
#     df = pd.read_sql_query(query, conn)

#     # Close the database connection
#     conn.close()

#     # Plot a bar graph
#     plt.figure(figsize=(10, 6))
#     df[column_to_plot].value_counts().plot(kind='bar', color='skyblue')
#     plt.title(f'Bar Graph of {column_to_plot}')
#     plt.xlabel(column_to_plot)
#     plt.ylabel('Count')
#     plt.show()


# # Plot a bar graph based on a specific column
# column_to_plot = 'column_name'  # Replace with the actual column name you want to plot
# plot_bar_graph(database_name, table_name, column_to_plot)

In [ ]:
df = pd.read_csv("Data/drugs_side_effects_drugs_com.csv")
df.head()

In [ ]:
# df.info()

In [ ]:
df = df.drop(['no_of_reviews', 'drug_link','medical_condition_url',"related_drugs"], axis = 1)
df.shape

In [ ]:
# Pie Plot

#figure size
sns.set_context('poster', font_scale=0.5)
plt.figure(figsize=(5,5))

df['rx_otc'].value_counts().plot.pie(autopct='%1.1f%%', textprops={'fontsize':12}).set_title("Medication distribution")

df['rx_otc'].value_counts()

In [ ]:
#Pie Plot
# df['medical_condition'].value_counts().plot.pie(autopct='%1.1f%%', textprops={'fontsize':12}).set_title("Target distribution on medical_condition")
top10_medicalCondition_df = df['medical_condition'].value_counts().head(10)
top10_medicalCondition_df

In [ ]:
plt.figure(figsize=(10, 6))
top10_medicalCondition_df.plot(kind='bar', color='skyblue')
plt.title('Top 10 Medical Conditions')
plt.xlabel('Medical Condition')
plt.ylabel('Count')
plt.show()

In [ ]:
# Set your AWS credentials and region
aws_access_key_id = aws_access_key_id
aws_secret_access_key = aws_secret_access_key 
region_name = "us-east-2"
bucket_name = "capstone-pill-images"
data_prefix = "data/data"

In [ ]:
def download_images_from_s3(bucket_name, prefix, local_directory):
    conf = SparkConf().setAppName("S3ImageDownloader")
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)

    # Function to download a single file from S3
    def download_file_from_s3(key):
        local_path = os.path.join(local_directory, os.path.relpath(key, prefix))
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        s3.download_file(bucket_name, key, local_path)

    # List objects from S3
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    keys = [obj['Key'] for obj in response.get('Contents', [])]

    # Create an RDD from the list of keys
    keys_rdd = sc.parallelize(keys)

    # Use Spark's map operation to parallelize the file downloads
    keys_rdd.foreach(download_file_from_s3)

    # Stop SparkContext
    sc.stop()